In [1]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from sklearn.metrics import (confusion_matrix, precision_score, auc,
                             roc_curve, recall_score, classification_report, f1_score)
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import random
import scipy.stats as stats
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import csv
from sklearn.cluster import KMeans
from sklearn_som.som import SOM
from sklearn import svm
from sklearn.metrics import auc
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import StratifiedKFold
from sklearn import tree
from sklearn.decomposition import PCA

# Dados NSL-KDD

In [2]:
data = pd.read_csv(r"C:\Users\luang\Downloads\BRACIS\UNSW_2018_IoT_Botnet_Full5p.csv")
x = data.drop (['category', 'subcategory', 'attack'], axis = 1)
y = data ['attack']

In [3]:
data = x.to_numpy()
data_class = y.to_numpy()

# Abordagem Autoencoder-KNN

In [4]:
epochs = 50
batch_size = 256
optimizer = Adam(learning_rate=0.0001)
activation = "relu"
neighbors = 11

num_folds = 10

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True)

fold_no = 1
count = 0

#Métricas ErroRecons
matrix_ErroRecons = np.zeros((num_folds,2,2))
accuracy_ErroRecons = np.zeros(num_folds)
recall_ErroRecons = np.zeros(num_folds)
precision_ErroRecons = np.zeros(num_folds)
f1_ErroRecons = np.zeros(num_folds)

#Métricas KNN
matrix_knn = np.zeros((num_folds,2,2))
accuracy_knn = np.zeros(num_folds)
recall_knn = np.zeros(num_folds)
precision_knn = np.zeros(num_folds)
f1_knn = np.zeros(num_folds)

#Métricas Decision Tree
matrix_dt = np.zeros((num_folds,2,2))
accuracy_dt = np.zeros(num_folds)
recall_dt = np.zeros(num_folds)
precision_dt = np.zeros(num_folds)
f1_dt = np.zeros(num_folds)

#Métricas SVM
matrix_svm = np.zeros((num_folds,2,2))
accuracy_svm = np.zeros(num_folds)
recall_svm = np.zeros(num_folds)
precision_svm = np.zeros(num_folds)
f1_svm = np.zeros(num_folds)

for train, test in kfold.split(data, data_class):
    print("Fold ",fold_no)
    #####################################################
    ####Pré-Processamento
    print("Pré-processando...")
    
    #Separação dos Folds de Treinamento e Teste
    dataTrain = data[train]
    dataTrain_class = data_class[train]
    modelTest = data[test]
    modelTest_class = data_class[test]
    
    #Conta Quantidade de dados
    Train = dataTrain.shape[1]
    noTrain = np.count_nonzero(dataTrain_class == 0)
    anTrain = np.count_nonzero(dataTrain_class == 1)

    #Separa dados normais e anômalos
    normalTrain = dataTrain[np.where(dataTrain_class == 0)]
    anomalyTrain = dataTrain[np.where(dataTrain_class == 1)]
    
    #Divide dados para treinamento
    porcen = 1
    j = int(noTrain * porcen)
    
    #Índices escolhidos aleatoriamente
    numbers_knn_normal = np.array(random.sample(range(0, noTrain),j))
    numbers_knn_anomaly = np.array(random.sample(range(0,anTrain),j))
    numbers_autoencoder_anomaly = np.array(list(set(np.arange(0, anTrain)) - set(numbers_knn_anomaly)))
    
    #Dados KNN
    knnTrain_normal = normalTrain[numbers_knn_normal]
    knnTrain_anomaly = anomalyTrain[numbers_knn_anomaly]
    knnTrain = np.concatenate((knnTrain_normal, knnTrain_anomaly), axis=0)
    knnTrain_class = np.concatenate((np.zeros(j), np.ones(j)))
    
    #Dados Autoencoder
    autoencoderTrain = anomalyTrain[numbers_autoencoder_anomaly]
    autoencoderTrain_class = np.ones(autoencoderTrain.shape[0])
    
    #Normalização
    scaler = MinMaxScaler()
    scaler.fit(knnTrain)
    autoencoderTrain = scaler.transform(autoencoderTrain)
    knnTrain = scaler.transform(knnTrain)
    modelTest = scaler.transform(modelTest)
    
    #####################################################
    ####PCA
    print("PCA")
    var = autoencoderTrain.shape[1]
    #Modela o PCA
    pca = PCA(n_components=int(var/4))    
    
    #####################################################
    ####KNN
    ###Treinamento KNN
    print("KNN...")
    pca.fit(autoencoderTrain)
    pca_encoder = pca.transform(knnTrain)
    neigh = KNeighborsClassifier(n_neighbors=neighbors)
    neigh.fit(pca_encoder, knnTrain_class)
    
    #Teste do Modelo
    pca_encoder = pca.transform(modelTest)
    predict_knn = neigh.predict(pca_encoder)
    
    #Métricas
    accuracy_knn[count] = accuracy_score(modelTest_class, predict_knn[:])
    recall_knn[count] = recall_score(modelTest_class, predict_knn[:])
    precision_knn[count] = precision_score(modelTest_class, predict_knn[:])
    f1_knn[count] = f1_score(modelTest_class, predict_knn[:])
    matrix_knn[count] = confusion_matrix(modelTest_class, predict_knn[:])
    print("Finalizado.")
   
    #####################################################
    ####Decision Tree
    ###Treinamento Decision Tree 
    print("DT...")
    pca_encoder = pca.transform(knnTrain)
    dt = tree.DecisionTreeClassifier()
    dt.fit(pca_encoder, knnTrain_class)

    #Teste
    pca_encoder = pca.transform(modelTest)
    predict_dt = dt.predict(pca_encoder)

    #Métricas
    accuracy_dt[count] = accuracy_score(modelTest_class, predict_dt[:])
    recall_dt[count] = recall_score(modelTest_class, predict_dt[:])
    precision_dt[count] = precision_score(modelTest_class, predict_dt[:])
    f1_dt[count] = f1_score(modelTest_class, predict_dt[:])
    matrix_dt[count] = confusion_matrix(modelTest_class, predict_dt[:])
    print("Finalizado.")
    
    #####################################################
    ####SVM
    #Treinamento SVM
    print("SVM...")
    pca_encoder = pca.transform(knnTrain)
    clf = svm.SVC(kernel='rbf')
    clf.fit(pca_encoder, knnTrain_class)
    
    #Teste
    pca_encoder = pca.transform(modelTest)
    predict_svm = clf.predict(pca_encoder)
    
    #Métricas
    accuracy_svm[count] = accuracy_score(modelTest_class, predict_svm[:])
    recall_svm[count] = recall_score(modelTest_class, predict_svm[:])
    precision_svm[count] = precision_score(modelTest_class, predict_svm[:])
    f1_svm[count] = f1_score(modelTest_class, predict_svm[:])
    matrix_svm[count] = confusion_matrix(modelTest_class, predict_svm[:])
    print("Finalizado.")
    
    count = count + 1
    fold_no = fold_no + 1

Fold  1
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  2
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  3
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  4
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  5
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  6
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  7
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  8
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  9
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.
Fold  10
Pré-processando...
PCA
KNN...
Finalizado.
DT...
Finalizado.
SVM...
Finalizado.


In [5]:
print("Autoencoder - KNN:\n", np.mean(matrix_knn, axis=0), "\nAcurácia: ", np.mean(accuracy_knn), ' +/- ', np.std(accuracy_knn), "\nRecall: ", np.mean(recall_knn), ' +/- ', np.std(recall_knn), "\nPrecision: ", np.mean(precision_knn), ' +/- ', np.std(precision_knn), "\nF1: ", np.mean(f1_knn), ' +/- ', np.std(f1_knn))

print("\nAutoencoder - SVM:\n", np.mean(matrix_svm, axis=0), "\nAcurácia: ", np.mean(accuracy_svm), ' +/- ', np.std(accuracy_svm), "\nRecall: ", np.mean(recall_svm), ' +/- ', np.std(recall_svm), "\nPrecision: ", np.mean(precision_svm), ' +/- ', np.std(precision_svm), "\nF1: ", np.mean(f1_svm), ' +/- ', np.std(f1_svm))

print("\nAutoencoder - Decision Tree:\n", np.mean(matrix_dt, axis=0), "\nAcurácia: ", np.mean(accuracy_dt), ' +/- ', np.std(accuracy_dt), "\nRecall: ", np.mean(recall_dt), ' +/- ', np.std(recall_dt), "\nPrecision: ", np.mean(precision_dt), ' +/- ', np.std(precision_dt), "\nF1: ", np.mean(f1_dt), ' +/- ', np.std(f1_dt))

Autoencoder - KNN:
 [[4.760000e+01 1.000000e-01]
 [6.132300e+03 3.606722e+05]] 
Acurácia:  0.9832837305137259  +/-  0.0038778840504680463 
Recall:  0.9832818280040808  +/-  0.0038782981517855673 
Precision:  0.9999997223571875  +/-  8.329284375552247e-07 
F1:  0.9915664604262376  +/-  0.0019708810688777267

Autoencoder - SVM:
 [[4.760000e+01 1.000000e-01]
 [5.524600e+03 3.612799e+05]] 
Acurácia:  0.9849402547172879  +/-  0.0038710942808877877 
Recall:  0.9849385686985606  +/-  0.0038717219676025227 
Precision:  0.9999997237019518  +/-  8.288941446887854e-07 
F1:  0.9924081715709867  +/-  0.0019657850029759405

Autoencoder - Decision Tree:
 [[4.710000e+01 6.000000e-01]
 [6.843700e+03 3.599608e+05]] 
Acurácia:  0.9813431655050634  +/-  0.00819190254671884 
Recall:  0.981342376614359  +/-  0.008194148537917369 
Precision:  0.9999983431569939  +/-  3.3055012579733537e-06 
F1:  0.9905652227946762  +/-  0.004182911769412653


In [6]:
Result = np.array(["KNN", "Acurácia", accuracy_knn, np.mean(accuracy_knn), np.std(accuracy_knn),"",
                                "Recall", recall_knn, np.mean(recall_knn), np.std(recall_knn), "",
                                "Precision", precision_knn, np.mean(precision_knn), np.std(precision_knn), "",
                                "F1", f1_knn, np.mean(f1_knn), np.std(f1_knn), "-----------------","-----------------",
                  "Decision Tree",  "Acurácia", accuracy_dt, np.mean(accuracy_dt), np.std(accuracy_dt),"",
                                "Recall", recall_dt, np.mean(recall_dt), np.std(recall_dt), "",
                                "Precision", precision_dt, np.mean(precision_dt), np.std(precision_dt), "",
                                "F1", f1_dt, np.mean(f1_dt), np.std(f1_dt), "-----------------","-----------------",
                  "SVM",  "Acurácia", accuracy_svm, np.mean(accuracy_svm), np.std(accuracy_svm),"",
                                "Recall", recall_svm, np.mean(recall_svm), np.std(recall_svm), "",
                                "Precision", precision_svm, np.mean(precision_svm), np.std(precision_svm), "",
                                "F1", f1_svm, np.mean(f1_svm), np.std(f1_svm), "","",])

np.savetxt('Result_PCA_IoT.txt', Result, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [7]:
Result_Matrix = np.array(["KNN", matrix_knn, np.mean(matrix_knn, axis=0), np.std(matrix_knn, axis=0), "-----------------","-----------------",
                          "Decision Tree", matrix_dt, np.mean(matrix_dt, axis=0), np.std(matrix_dt, axis=0), "-----------------","-----------------",
                          "SVM", matrix_svm, np.mean(matrix_svm, axis=0), np.std(matrix_svm, axis=0)])

np.savetxt('Result_PCA_Matrix_IoT.txt', Result_Matrix, delimiter = ',',fmt='%s')

C:\Users\luang\anaconda3\envs\keras_env\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
